In [1]:
#importing dataset
import pandas as pd
df= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\train.csv")
len(df)

31962

In [2]:
#preprocessing 
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import nltk
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize
import string

df['new_tweet'] = df.tweet.str.replace('@user', '')
df['new_tweet'] = df['new_tweet'].str.replace("[^a-zA-Z#]", " " )
df['new_tweet'] = df['new_tweet'].str.replace("#", " ")
df['new_tweet'] = df['new_tweet'].apply(lambda x: x.split())
df['new_tweet']= df['new_tweet'].apply(lambda x: [stemmer.stem(i) for i in x])
def process(text):
    # Check characters to see if they are in punctuation
    nopunc = set(char for char in list(text) if char not in string.punctuation)
    # Join the characters to form the string.
    nopunc = " ".join(nopunc)
    # remove any stopwords if present
    return [word for word in nopunc.lower().split() if word.lower() not in stopwords]
df['new_tweet'] = df['new_tweet'].apply(process)
df=df.drop(columns='tweet')
df=df.drop(columns='id')
df.head()

<ipython-input-2-d574088bc6ff>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['new_tweet'] = df['new_tweet'].str.replace("[^a-zA-Z#]", " " )


,label,new_tweet
0,0,"[selfish, drag, dysfunct, run, kid, father]"
1,0,"[lyft, van, getthank, use, disapoint, credit, ..."
2,0,"[bihday, majesti]"
3,0,"[time, ur, model, u, take, love]"
4,0,"[motiv, factsguid, societi]"


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
x_train, x_test, y_train, y_test =train_test_split(df["new_tweet"], df["label"], test_size = 0.2, random_state = 42)

In [4]:
import numpy as np
N = 10    
split=np.array_split(x_train, N)

In [5]:
import numpy as np
N = 10    
label=np.array_split(y_train, N)

In [6]:
from modAL.models import ActiveLearner
from sklearn.ensemble import RandomForestClassifier
from modAL.uncertainty import entropy_sampling, uncertainty_sampling, margin_sampling
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(split[0].apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
print("Number of Training samples : ",len(split[0]))
learner = ActiveLearner(estimator=RandomForestClassifier(),query_strategy=entropy_sampling,X_training=x_train_tfidf, y_training=label[0].ravel())
print("Number of Testing samples : ",len(x_test))
x_test_counts = count_vect.transform(x_test.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions = learner.predict(x_test_tfidf)
print(accuracy_score(y_test,predictions)*100)


Number of Training samples :  2557
Number of Testing samples :  6393
94.32191459408729


In [7]:
x_test_counts = count_vect.transform(x_test.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions = learner.predict(x_test_tfidf)
print(accuracy_score(y_test,predictions)*100)

94.32191459408729


In [8]:
#Queriying Using ModAL
for i in range(1,10):
    x_counts = count_vect.transform(split[i].apply(lambda x: ' '.join(x)))
    x_tfidf = transformer.transform(x_counts)
    #for j in range(100):
    query_idx, query_sample = learner.query(x_tfidf, n_instances=200)
    split[i]=split[i].reset_index()
    split[i]=split[i].copy()
    split[i]=split[i].drop(['index'],axis=1)
    #print(split[i].loc[query_idx])
    #y=int(input("Enter the label"))
    #y=label[i].iloc[query_idx]
    #y=y.item()
    y=[]
    #Teaching 
    for k in query_idx:
      j=label[i].iloc[k]
      lb=j.item()
      y.append(lb)
    #Teaching using ModAL
    learner.teach(x_tfidf[query_idx],y)      
x_test_counts = count_vect.transform(x_test.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions = learner.predict(x_test_tfidf)
print(accuracy_score(y_test,predictions)*100)

95.66713592992335
